In [1]:
cd ..

/Users/muhammadfarooq/Codebase/thesis-experiment/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_sixteen_url = "damage/damage_sixteen.csv"
damage_thirtytwo_url = "damage/damage_thirtytwo.csv"

df_damage_sixteen_min_max = read_damage_min_max(damage_sixteen_url)
df_damage_thirtytwo_min_max = read_damage_min_max(damage_thirtytwo_url)

df_damage_sixteen_mean = read_damage_mean(damage_sixteen_url)
df_damage_thirtytwo_mean = read_damage_mean(damage_thirtytwo_url)

In [6]:
#df_damage_sixteen_min_max

In [7]:
#df_damage_thirtytwo_min_max

In [8]:
#df_damage_sixteen_mean

In [9]:
#df_damage_thirtytwo_mean

In [10]:
#STOP

In [11]:
df = df_damage_sixteen_min_max

In [12]:
#df

In [13]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [14]:
#STOP

# Config

In [15]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [16]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [17]:
units = [128]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [18]:
#STOP

# Model

In [19]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [20]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [21]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [22]:
input_dim = X_train.shape[1:]

In [ ]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 128 Activation: relu Kernel size: 5 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 4s - loss: 7.2775 - accuracy: 0.4318 - val_loss: 14.7335 - val_accuracy: 0.3556 - 4s/epoch - 1s/step
Epoch 2/50
3/3 - 2s - loss: 5.9081 - accuracy: 0.5795 - val_loss: 3.4130 - val_accuracy: 0.7556 - 2s/epoch - 813ms/step
Epoch 3/50
3/3 - 3s - loss: 2.0326 - accuracy: 0.8523 - val_loss: 1.2885 - val_accuracy: 0.8444 - 3s/epoch - 858ms/step
Epoch 4/50
3/3 - 2s - loss: 0.9454 - accuracy: 0.8409 - val_loss: 0.4959 - val_accuracy: 0.8000 - 2s/epoch - 812ms/step
Epoch 5/50
3/3 - 2s - loss: 0.4407 - accuracy: 0.8750 - val_loss: 0.5123 - val_accuracy: 0.7778 - 2s/epoch - 747ms/step
Epoch 6/50
3/3 - 2s - loss: 0.3548 - accuracy: 0.8977 - val_loss: 0.3633 - val_accuracy: 0.8667 - 2s/epoch - 809ms/step
Epoch 7/50
3/3 - 2s - loss: 0.2646 - accuracy: 0.8864 - val_loss: 0.5120 - val_accuracy: 0.7556 - 2s/epoch - 823ms/step
Epoch 8/50
3/3 - 3s - loss: 0.2417 - accuracy: 0.8864 - val_loss: 0.

Epoch 12/50
3/3 - 2s - loss: 0.0930 - accuracy: 0.9773 - val_loss: 0.3442 - val_accuracy: 0.8222 - 2s/epoch - 749ms/step
Epoch 13/50
3/3 - 2s - loss: 0.0716 - accuracy: 0.9886 - val_loss: 0.3654 - val_accuracy: 0.8667 - 2s/epoch - 750ms/step
Epoch 14/50
3/3 - 2s - loss: 0.0644 - accuracy: 0.9886 - val_loss: 0.5093 - val_accuracy: 0.8000 - 2s/epoch - 749ms/step
Epoch 15/50
3/3 - 2s - loss: 0.0507 - accuracy: 1.0000 - val_loss: 0.3807 - val_accuracy: 0.8667 - 2s/epoch - 752ms/step
Epoch 16/50
3/3 - 2s - loss: 0.0844 - accuracy: 0.9659 - val_loss: 0.3341 - val_accuracy: 0.8667 - 2s/epoch - 748ms/step
Epoch 17/50
3/3 - 2s - loss: 0.0264 - accuracy: 1.0000 - val_loss: 0.6616 - val_accuracy: 0.8222 - 2s/epoch - 743ms/step
Epoch 18/50
3/3 - 2s - loss: 0.0575 - accuracy: 0.9886 - val_loss: 0.3726 - val_accuracy: 0.8667 - 2s/epoch - 738ms/step
Epoch 19/50
3/3 - 2s - loss: 0.0195 - accuracy: 1.0000 - val_loss: 0.4698 - val_accuracy: 0.8667 - 2s/epoch - 761ms/step
Epoch 20/50
3/3 - 2s - loss: 0.0

Epoch 5/50
3/3 - 2s - loss: 0.4926 - accuracy: 0.8539 - val_loss: 0.5400 - val_accuracy: 0.8182 - 2s/epoch - 819ms/step
Epoch 6/50
3/3 - 2s - loss: 0.2764 - accuracy: 0.9101 - val_loss: 0.5652 - val_accuracy: 0.8182 - 2s/epoch - 760ms/step
Epoch 7/50
3/3 - 2s - loss: 0.2179 - accuracy: 0.8989 - val_loss: 0.7688 - val_accuracy: 0.8182 - 2s/epoch - 756ms/step
Epoch 8/50
3/3 - 2s - loss: 0.2422 - accuracy: 0.8989 - val_loss: 0.7401 - val_accuracy: 0.8182 - 2s/epoch - 758ms/step
Epoch 9/50
3/3 - 2s - loss: 0.1720 - accuracy: 0.9213 - val_loss: 0.6596 - val_accuracy: 0.8182 - 2s/epoch - 751ms/step
Epoch 10/50
3/3 - 2s - loss: 0.1493 - accuracy: 0.9213 - val_loss: 0.7185 - val_accuracy: 0.8182 - 2s/epoch - 758ms/step
Epoch 11/50
3/3 - 2s - loss: 0.1263 - accuracy: 0.9551 - val_loss: 0.9538 - val_accuracy: 0.8409 - 2s/epoch - 760ms/step
Epoch 12/50
3/3 - 2s - loss: 0.1897 - accuracy: 0.9775 - val_loss: 0.8379 - val_accuracy: 0.7727 - 2s/epoch - 759ms/step
Epoch 13/50
3/3 - 2s - loss: 0.1827 -

Epoch 39/50
3/3 - 2s - loss: 0.0278 - accuracy: 1.0000 - val_loss: 0.2678 - val_accuracy: 0.8667 - 2s/epoch - 741ms/step
Epoch 40/50
3/3 - 2s - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.2708 - val_accuracy: 0.8667 - 2s/epoch - 740ms/step
Epoch 41/50
3/3 - 2s - loss: 0.0451 - accuracy: 0.9886 - val_loss: 0.2914 - val_accuracy: 0.8667 - 2s/epoch - 747ms/step
Epoch 42/50
3/3 - 2s - loss: 0.0396 - accuracy: 1.0000 - val_loss: 0.3840 - val_accuracy: 0.8667 - 2s/epoch - 746ms/step
Epoch 1/50
3/3 - 3s - loss: 23.1783 - accuracy: 0.4157 - val_loss: 2.9164 - val_accuracy: 0.5000 - 3s/epoch - 1s/step
Epoch 2/50
3/3 - 3s - loss: 2.0085 - accuracy: 0.5056 - val_loss: 0.5861 - val_accuracy: 0.8409 - 3s/epoch - 949ms/step
Epoch 3/50
3/3 - 2s - loss: 0.4264 - accuracy: 0.8427 - val_loss: 2.2065 - val_accuracy: 0.5000 - 2s/epoch - 758ms/step
Epoch 4/50
3/3 - 2s - loss: 0.8366 - accuracy: 0.6966 - val_loss: 0.8369 - val_accuracy: 0.7727 - 2s/epoch - 729ms/step
Epoch 5/50
3/3 - 2s - loss: 0.3759 - a

Epoch 8/50
3/3 - 2s - loss: 0.4169 - accuracy: 0.8315 - val_loss: 0.3503 - val_accuracy: 0.9091 - 2s/epoch - 724ms/step
Epoch 9/50
3/3 - 2s - loss: 0.3891 - accuracy: 0.8427 - val_loss: 0.5595 - val_accuracy: 0.7955 - 2s/epoch - 718ms/step
Epoch 10/50
3/3 - 2s - loss: 0.5143 - accuracy: 0.7528 - val_loss: 0.3302 - val_accuracy: 0.9091 - 2s/epoch - 721ms/step
Epoch 11/50
3/3 - 2s - loss: 0.3622 - accuracy: 0.8539 - val_loss: 0.3506 - val_accuracy: 0.9091 - 2s/epoch - 723ms/step
Epoch 12/50
3/3 - 2s - loss: 0.3123 - accuracy: 0.8652 - val_loss: 0.3070 - val_accuracy: 0.9091 - 2s/epoch - 773ms/step
Epoch 13/50
3/3 - 2s - loss: 0.2929 - accuracy: 0.8764 - val_loss: 0.4238 - val_accuracy: 0.8864 - 2s/epoch - 734ms/step
Epoch 14/50
3/3 - 2s - loss: 0.3410 - accuracy: 0.8876 - val_loss: 0.3326 - val_accuracy: 0.9091 - 2s/epoch - 735ms/step
Epoch 15/50
3/3 - 2s - loss: 0.3448 - accuracy: 0.8539 - val_loss: 0.3431 - val_accuracy: 0.9091 - 2s/epoch - 739ms/step
Epoch 16/50
3/3 - 2s - loss: 0.304

...

Epoch 38/50
3/3 - 2s - loss: 0.2565 - accuracy: 0.8864 - val_loss: 0.4545 - val_accuracy: 0.7556 - 2s/epoch - 749ms/step
Epoch 39/50
3/3 - 2s - loss: 0.2849 - accuracy: 0.8864 - val_loss: 0.4207 - val_accuracy: 0.7778 - 2s/epoch - 747ms/step
Epoch 1/50
3/3 - 3s - loss: 2.0687 - accuracy: 0.4607 - val_loss: 0.5191 - val_accuracy: 0.8409 - 3s/epoch - 1s/step
Epoch 2/50
3/3 - 3s - loss: 0.7356 - accuracy: 0.6854 - val_loss: 0.6473 - val_accuracy: 0.7500 - 3s/epoch - 887ms/step
Epoch 3/50
3/3 - 2s - loss: 0.3413 - accuracy: 0.8876 - val_loss: 0.5361 - val_accuracy: 0.7955 - 2s/epoch - 719ms/step
Epoch 4/50
3/3 - 2s - loss: 0.3799 - accuracy: 0.8539 - val_loss: 0.7834 - val_accuracy: 0.7045 - 2s/epoch - 719ms/step
Epoch 5/50
3/3 - 2s - loss: 0.4027 - accuracy: 0.8539 - val_loss: 0.6523 - val_accuracy: 0.7500 - 2s/epoch - 724ms/step
Epoch 6/50
3/3 - 2s - loss: 0.3634 - accuracy: 0.8876 - val_loss: 0.5406 - val_accuracy: 0.8182 - 2s/epoch - 724ms/step
Epoch 7/50
3/3 - 2s - loss: 0.3577 - accu

Epoch 42/50
3/3 - 2s - loss: 0.0391 - accuracy: 1.0000 - val_loss: 0.3766 - val_accuracy: 0.8667 - 2s/epoch - 766ms/step
Epoch 43/50
3/3 - 2s - loss: 0.0371 - accuracy: 1.0000 - val_loss: 0.3784 - val_accuracy: 0.8444 - 2s/epoch - 764ms/step
Epoch 44/50
3/3 - 2s - loss: 0.0357 - accuracy: 1.0000 - val_loss: 0.3847 - val_accuracy: 0.8444 - 2s/epoch - 767ms/step
Epoch 1/50
3/3 - 4s - loss: 0.8631 - accuracy: 0.5618 - val_loss: 0.7897 - val_accuracy: 0.5909 - 4s/epoch - 1s/step
Epoch 2/50
3/3 - 2s - loss: 0.4623 - accuracy: 0.8090 - val_loss: 0.4935 - val_accuracy: 0.8409 - 2s/epoch - 831ms/step
Epoch 3/50
3/3 - 2s - loss: 0.3716 - accuracy: 0.8764 - val_loss: 0.5440 - val_accuracy: 0.8182 - 2s/epoch - 782ms/step
Epoch 4/50
3/3 - 2s - loss: 0.3577 - accuracy: 0.8989 - val_loss: 0.7494 - val_accuracy: 0.7727 - 2s/epoch - 787ms/step
Epoch 5/50
3/3 - 2s - loss: 0.3311 - accuracy: 0.8876 - val_loss: 0.5386 - val_accuracy: 0.8409 - 2s/epoch - 779ms/step
Epoch 6/50
3/3 - 2s - loss: 0.4938 - acc

Epoch 21/50
3/3 - 2s - loss: 0.1592 - accuracy: 0.9091 - val_loss: 0.3758 - val_accuracy: 0.8444 - 2s/epoch - 789ms/step
Epoch 22/50
3/3 - 2s - loss: 0.1370 - accuracy: 0.9545 - val_loss: 0.3796 - val_accuracy: 0.8444 - 2s/epoch - 782ms/step
Epoch 23/50
3/3 - 2s - loss: 0.1409 - accuracy: 0.9545 - val_loss: 0.3706 - val_accuracy: 0.8444 - 2s/epoch - 799ms/step
Epoch 24/50
3/3 - 2s - loss: 0.1199 - accuracy: 0.9659 - val_loss: 0.3639 - val_accuracy: 0.8667 - 2s/epoch - 790ms/step
Epoch 25/50
3/3 - 2s - loss: 0.1041 - accuracy: 0.9773 - val_loss: 0.3749 - val_accuracy: 0.8444 - 2s/epoch - 791ms/step
Epoch 26/50
3/3 - 2s - loss: 0.1055 - accuracy: 0.9545 - val_loss: 0.3787 - val_accuracy: 0.8667 - 2s/epoch - 784ms/step
Epoch 27/50
3/3 - 2s - loss: 0.0968 - accuracy: 0.9773 - val_loss: 0.4047 - val_accuracy: 0.8444 - 2s/epoch - 781ms/step
Epoch 28/50
3/3 - 2s - loss: 0.0917 - accuracy: 0.9773 - val_loss: 0.4586 - val_accuracy: 0.8000 - 2s/epoch - 791ms/step
Epoch 1/50
3/3 - 4s - loss: 0.67

Epoch 30/50
3/3 - 2s - loss: 0.1075 - accuracy: 0.9886 - val_loss: 0.3721 - val_accuracy: 0.8444 - 2s/epoch - 829ms/step
Epoch 31/50
3/3 - 3s - loss: 0.1137 - accuracy: 0.9773 - val_loss: 0.3692 - val_accuracy: 0.8667 - 3s/epoch - 837ms/step
Epoch 32/50
3/3 - 2s - loss: 0.1129 - accuracy: 0.9659 - val_loss: 0.3646 - val_accuracy: 0.8444 - 2s/epoch - 830ms/step
Epoch 33/50
3/3 - 3s - loss: 0.0992 - accuracy: 0.9659 - val_loss: 0.3740 - val_accuracy: 0.8444 - 3s/epoch - 844ms/step
Epoch 34/50
3/3 - 3s - loss: 0.0969 - accuracy: 0.9886 - val_loss: 0.4214 - val_accuracy: 0.8000 - 3s/epoch - 835ms/step
Epoch 35/50
3/3 - 2s - loss: 0.0935 - accuracy: 0.9886 - val_loss: 0.4000 - val_accuracy: 0.8222 - 2s/epoch - 832ms/step
Epoch 36/50
3/3 - 2s - loss: 0.0926 - accuracy: 0.9659 - val_loss: 0.3855 - val_accuracy: 0.8444 - 2s/epoch - 829ms/step
Epoch 37/50
3/3 - 3s - loss: 0.0746 - accuracy: 1.0000 - val_loss: 0.3825 - val_accuracy: 0.8222 - 3s/epoch - 834ms/step
Epoch 38/50
3/3 - 3s - loss: 0.0

Epoch 18/50
3/3 - 2s - loss: 0.3388 - accuracy: 0.8864 - val_loss: 0.4098 - val_accuracy: 0.8000 - 2s/epoch - 710ms/step
Epoch 19/50
3/3 - 2s - loss: 0.3269 - accuracy: 0.8750 - val_loss: 0.4040 - val_accuracy: 0.8444 - 2s/epoch - 710ms/step
Epoch 20/50
3/3 - 2s - loss: 0.3056 - accuracy: 0.8864 - val_loss: 0.4081 - val_accuracy: 0.8000 - 2s/epoch - 713ms/step
Epoch 21/50
3/3 - 2s - loss: 0.3127 - accuracy: 0.8864 - val_loss: 0.4059 - val_accuracy: 0.8222 - 2s/epoch - 718ms/step
Epoch 22/50
3/3 - 2s - loss: 0.3061 - accuracy: 0.8864 - val_loss: 0.3989 - val_accuracy: 0.8444 - 2s/epoch - 750ms/step
Epoch 23/50
3/3 - 2s - loss: 0.2924 - accuracy: 0.8864 - val_loss: 0.3983 - val_accuracy: 0.8444 - 2s/epoch - 747ms/step
Epoch 24/50
3/3 - 2s - loss: 0.3011 - accuracy: 0.8864 - val_loss: 0.3978 - val_accuracy: 0.8444 - 2s/epoch - 763ms/step
Epoch 25/50
3/3 - 2s - loss: 0.2978 - accuracy: 0.8864 - val_loss: 0.4170 - val_accuracy: 0.8000 - 2s/epoch - 724ms/step
Epoch 26/50
3/3 - 2s - loss: 0.2

In [ ]:
print(best_acc)
print(baseline_values)

In [ ]:
#STOP